In [1]:
#--------IMPORTANTE----- IMPORTAR FUNCIONES DESDE VISUAL CODE.
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import matplotlib.ticker as mticker
import sys
sys.path.append('../')
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2
import plotly.express as px

In [2]:

shark = pd.read_csv("data/attacks.csv",encoding = "ISO-8859-1")

In [3]:
shark_cop = shark.copy()

In [4]:
shark.shape

(25723, 24)

In [5]:
shark.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [6]:
shark.sample()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
20570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
shark.dropna(axis=0, how = "all", inplace=True) #Elimino todas las filas que sus valores sean nan.

In [8]:
shark.dropna(axis=1, how = "all", inplace=True) #Elimino las columnas en las que todos sus valores sean Nan

In [9]:
shark.shape #EL 'RAW' Data Frame tenía 25723 filas, una vez eliminadas todas las filas en las que sus valores eran Nan, quedan 8703, por lo tanto se han borrado 17020 filas en las que todos sus valores eran nulos.

(8703, 24)

In [10]:
shark.sample()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
4223,1956.12.24,24-Dec-1956,1956.0,Unprovoked,PAPUA NEW GUINEA,Milne Bay Province,Samarai Island,Spearfishing,Titus Tiso,M,...,NaN,"Beavis, Kwato Mission; A. M. Rapson, p. 149; V...",1956.12.24-Tiso.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1956.12.24,1956.12.24,2080.0,NaN,NaN


## Posibles hipótesis a desarrollar

#Hipótesis 1: Cual es la relacion entre muertes de hombres y cuantas mujeres. 
###interests columns: sex,fatal(y/n)

#Hipótesis 2: Como ha desarrollado el numero de muertes (desde que hay infromacion) en los difrentes paises o continentes?
### interest columns : year, country, fatal(y/n) 

#Hipótesis 3: A manos de que especie causa mas muertes.
### interest columns : species, fatal(y/n) 

#Hipótesis 4: Cuantas de las personas que mueren a manos de los tiburones eran trabajadores y cuantas eran civiles.
### interest columns : activity, fatal(y/n)

#Hipótesis 5: Cual es la edad y sexo (edad media, la desviacion estandar, el maximo y el mínimo) en el que son atacados (Muertes y injury)
### interest columns : age, sex, fatal(y/n)



In [11]:
#Elimino las columnas que no influyen en el desarrollo de las hipótesis.
shark.drop(['Unnamed: 22','Unnamed: 23','original order','Case Number.2','Case Number.1','href','href formula','pdf','Investigator or Source','Time','Area','Location','Name','Case Number.1', 'Case Number'], axis=1 ,inplace=True)





In [12]:
shark.shape

(8703, 10)

In [13]:
shark.dropna(how = "all", inplace=True) #Vuelo a hacer dropna all, para eliminar todos las filas con valores nan.

In [14]:
shark.shape

(6302, 10)

In [15]:
shark.head(3)

,Date,Year,Type,Country,Activity,Sex,Age,Injury,Fatal (Y/N),Species
0,25-Jun-2018,2018.0,Boating,USA,Paddling,F,57,"No injury to occupant, outrigger canoe and pad...",N,White shark
1,18-Jun-2018,2018.0,Unprovoked,USA,Standing,F,11,Minor injury to left thigh,N,NaN
2,09-Jun-2018,2018.0,Invalid,USA,Surfing,M,48,Injury to left lower leg from surfboard skeg,N,NaN


In [16]:
#Atributo de intancia.
#Renombrar columnas, machacando el atributo de intancia.
shark.columns

Index(['Date', 'Year', 'Type', 'Country', 'Activity', 'Sex ', 'Age', 'Injury',
       'Fatal (Y/N)', 'Species '],
      dtype='object')

In [17]:
 #Creo una lista de las columnas y convertire todas dus letras en minuscula y sustituiré los espacios con barra baja
lista_sh = [sh.replace(' ','_').lower() for sh in shark.columns]

In [18]:
#Sustituyo las columnas existentes en el DF por las editadas en las lista, lista_sh
shark.columns = lista_sh

In [19]:
shark.sample()

,date,year,type,country,activity,sex_,age,injury,fatal_(y/n),species_
263,14-Jun-2016,2016.0,Unprovoked,USA,Floating in tube,F,6,Injury to lower leg,N,3' to 4' shark


In [20]:
# Escribir funcion para visualizar el porcentaje:
#tenemos esta pero mejor la de sergio!!!!
def dirt(x):
    x = x[0]
    porcentaje = (x*100)/shark.shape[0] #Visualizar columna.
    return f'The percentage of Nan values in this column is of, {round(porcentaje,2)}%.'

In [21]:
shark.select_dtypes(include =('float','int')).head()

,year
0,2018.0
1,2018.0
2,2018.0
3,2018.0
4,2018.0


In [22]:
shark.select_dtypes(include = 'object').sample(5)

,date,type,country,activity,sex_,age,injury,fatal_(y/n),species_
4910,25-Apr-1935,Invalid,AUSTRALIA,"Disappeared 11 days earlier, probable homicide...",M,40,Captive tiger shark regurgitated his arm in th...,NaN,Invalid
5182,18-Jun-1924,Provoked,USA,Removing shark from a net,M,NaN,Netted shark made a5-inch incision above the k...,N,NaN
5947,Reported 27-Jan-1864,Unprovoked,PANAMA,Coming ashore on a hawser,M,NaN,FATAL,Y,NaN
316,23-Jan-2016,Unprovoked,USA,Paddle boarding,M,48,No injury,N,"Tiger shark, 14'"
5377,06-Jan-1912,Unprovoked,AUSTRALIA,NaN,M,NaN,Thigh & lower abdomen severely bitten,N,C. macrurus captured 48 hours after attack wit...


In [23]:
shark.date.unique() 

array(['25-Jun-2018', '18-Jun-2018', '09-Jun-2018', ..., '1900-1905',
       '1883-1889', '1845-1853'], dtype=object)

In [24]:
#Cambiamos nombre a la columna fatal(n/y), que daba fallo al cargarla
shark.rename(columns={'fatal_(y/n)': 'fatal_YN'}, inplace=True)

In [25]:
#Teniendo en cuenta las posibles hipótesis formuladas, para desarrollar, escogeré aquellas en las que sus datos tengan menos valores Nan, para que las conclusiones sean mas precisas.

In [26]:
## Posibles hipótesis a desarrollar

#Hipótesis 1: Cual es la relacion entre muertes de hombres y cuantas mujeres. 
###interests columns: sex,fatal(y/n) - SI

#Hipótesis 2: Como ha desarrollado el numero de muertes (desde que hay infromacion) en los difrentes paises o continentes?
### interest columns : year, country (Australia y USA, poner condicion, con los paises que me interesen), fatal(y/n) - SI

#Hipótesis 3: A manos de que especie causa mas muertes.
### interest columns : species, fatal(y/n) - DESCARTADA

#Hipótesis 4: Cuantas de las personas que mueren a manos de los tiburones eran pescadores, surfistas o bañistas.
### interest columns : activity, fatal(y/n) - 

#Hipótesis 5: Cual es la edad y sexo (edad media, la desviacion estandar, el maximo y el mínimo) en el que son atacados (Muertes y injury)
### interest columns : age, sex, fatal(y/n) - DESCARTADA



In [27]:
shark.sample()

,date,year,type,country,activity,sex_,age,injury,fatal_YN,species_
4148,10-Jul-1958,1958.0,Unprovoked,USA,NaN,M,35,FATAL Autopsy report: bitten by shark while s...,Y,NaN


In [28]:
#Elimino las columnas:
# - species_, hay un 45.3% de datos nulos, que no pueden ser reemplazados.
#- age, hay un 44.92% de datos nulos que no pueden ser reemplazados.
#-date, tiene 0.0% de datos nulos, pero los datos no son claros, para desarrollar mi hipótesis, no necesito date, por ello me quedo con year.
#-Injury, aunque tiene un 0.4% de datos nulos, no es relevante para las hiposis a desarrollar, puesto que con las columna de fatal_YN, se puede apreciar si fue muerte o no. 
shark.drop(['species_','age','date','injury'], axis=1 ,inplace=True) 

In [29]:
shark.sample()

,year,type,country,activity,sex_,fatal_YN
4860,1937.0,Unprovoked,AUSTRALIA,NaN,M,N


In [30]:
shark.shape

(6302, 6)

In [31]:
shark.dropna(axis=0, how = "all", inplace=True)

In [32]:
shark.shape

(6302, 6)

In [33]:
#LIMPIEZA POR COLUMNAS:


In [34]:
#Comienzo por la columna que clasifica los ataques de mortales o no, exploro los valores que no estan claros.

shark.fatal_YN.value_counts()

N          4293
Y          1388
UNKNOWN      71
 N            7
M             1
2017          1
N             1
y             1
Name: fatal_YN, dtype: int64

In [35]:
shark.fatal_YN.isna().sum()

539

In [36]:
shark.loc[(shark["fatal_YN"] == '2017')]

,year,type,country,activity,sex_,fatal_YN
786,2012.0,Provoked,ITALY,Attempting to rescue an injured & beached shark,M,2017


In [37]:
shark.loc[(shark["fatal_YN"] == 'UNKNOWN')] #Demasiados casos sin confirmar de tiburon 

,year,type,country,activity,sex_,fatal_YN
624,2013.0,Unprovoked,USA,NaN,F,UNKNOWN
1270,2008.0,Unprovoked,AUSTRALIA,NaN,M,UNKNOWN
2250,1997.0,Unprovoked,BRAZIL,Bathing,M,UNKNOWN
2260,1997.0,Unprovoked,USA,NaN,M,UNKNOWN
2267,1996.0,Unprovoked,AUSTRALIA,Surfing,M,UNKNOWN
...,...,...,...,...,...,...
6167,1617.0,Unprovoked,INDIA,NaN,NaN,UNKNOWN
6169,1595.0,Unprovoked,INDIA,Ship lay at anchor & man was working on its ru...,M,UNKNOWN
6171,1555.0,Unprovoked,NaN,Swimming,M,UNKNOWN
6172,1554.0,Unprovoked,FRANCE,NaN,M,UNKNOWN


In [38]:
shark.loc[(shark["fatal_YN"] == ' N')]

,year,type,country,activity,sex_,fatal_YN
3942,1960.0,Unprovoked,AUSTRALIA,NaN,F,N
3944,1960.0,Unprovoked,AUSTRALIA,"Fishing, when line became snagged on rock & he...",M,N
3946,1960.0,Unprovoked,AUSTRALIA,Swimming,M,N
3948,1960.0,Provoked,AUSTRALIA,Chasing shark out of bathing area while riding...,M,N
3949,1960.0,Unprovoked,AUSTRALIA,"Spearfishing, speared fish retreated to cave w...",M,N
3951,1960.0,Provoked,NEW GUINEA,NaN,M,N
3956,1960.0,Unprovoked,PAPUA NEW GUINEA,Collecting shells,M,N


In [39]:
shark.loc[(shark["fatal_YN"] == 'N ')]

,year,type,country,activity,sex_,fatal_YN
5427,1909.0,Unprovoked,AUSTRALIA,NaN,NaN,N


In [40]:
shark.loc[(shark["fatal_YN"] == 'y')]


,year,type,country,activity,sex_,fatal_YN
6117,1815.0,Unprovoked,TOBAGO,Bathing,M,y


In [41]:
shark[shark.isna().any(axis=1)].sample(3)

,year,type,country,activity,sex_,fatal_YN
5061,1929.0,Unprovoked,AUSTRALIA,NaN,M,Y
3033,1981.0,Sea Disaster,BERMUDA,Foundering of the Israeli freighter Mezada,NaN,N
5321,1915.0,Unprovoked,USA,NaN,M,N


In [42]:
#Una vez observados los valores, procedemos a editar cambiamos en el data frame. Los valores que difieren de 'N','Y', seran convertidos en valores 'NAN' np.nan.
#Valores como 'y'& ' N', serán editados en sus respectivos.
shark['fatal_YN'].replace([' N', 'N ' ],'N', inplace=True)
shark['fatal_YN'].replace('y', 'Y', inplace=True)
shark['fatal_YN'].replace(['M','2017','UNKNOWN'],np.nan, inplace=True)

In [43]:
shark.fatal_YN.value_counts()

N    4301
Y    1389
Name: fatal_YN, dtype: int64

In [44]:
shark.fatal_YN.isna().sum()

612

In [45]:
#Exploring and filtering sex column.
shark.sex_.isna().sum()

565

In [46]:
shark.loc[(shark["sex_"] == 'M ')] #M

,year,type,country,activity,sex_,fatal_YN
563,2014.0,Unprovoked,USA,Swimming,M,N
1587,2005.0,Unprovoked,AUSTRALIA,Spearfishing,M,N


In [47]:
#Como en el DF original he borrado la columna 'name', utilizo la copia para ver si hay algun indicio de su sexo.
shark.loc[(shark["sex_"] == 'N')]


,year,type,country,activity,sex_,fatal_YN
4938,1934.0,Boating,AUSTRALIA,Fishing,N,N
6131,1801.0,Provoked,NaN,Standing on landed shark's tail,N,Y


In [48]:
#Busco sus nombres en internet, y veo que es un varon (Male) que murio a manos de un tiburon de 12' (https://sharkattackfile.net/spreadsheets/pdf_directory/1801.12.18.R-Pettigrew.pdf)
# y por otro lado, William & Leslie Newton: 2 varones(Male) (https://sharkattackfile.net/spreadsheets/pdf_directory/1934.07.11-Newton-boat-Australia.pdf)
shark_cop.loc[(shark_cop["Sex "] == 'N')]# M

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
4938,1934.07.11,11-Jul-1934,1934.0,Boating,AUSTRALIA,New South Wales,Cronulla,Fishing,"18' boat, occupants William & Leslie Newton",N,...,"Blue pointer, 11'","G.P. Whitley, ref: Daily Telegraph, 7/11/1934 ...",1934.07.11-Newton-boat-Australia.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1934.07.11,1934.07.11,1365.0,NaN,NaN
6131,1801.12.18.R,Reported 18-Dec-1801,1801.0,Provoked,NaN,NaN,NaN,Standing on landed shark's tail,Stephen Pettigew,N,...,12' shark,"The Evening Post, 12/18/1801",1801.12.18.R-Pettigrew.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1801.12.18.R,1801.12.18.R,172.0,NaN,NaN


In [49]:
shark.loc[(shark["sex_"] == 'lli')]

,year,type,country,activity,sex_,fatal_YN
1624,2004.0,Unprovoked,USA,Surfing,lli,N


In [50]:
shark_cop.loc[(shark_cop["Sex "] == 'lli')] # M


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
1624,2004.11.11.b,11-Nov-2004,2004.0,Unprovoked,USA,California,"Bunkers, Humboldt Bay, Eureka, Humboldt County",Surfing,Brian Kang,lli,...,5.5 m [18'] white shark,"R. Collier, GSAF",2004.11.11.b-Kang.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2004.11.11.b,2004.11.11.b,4679.0,NaN,NaN


In [51]:
#Trato de comprobar si es M o F
shark.loc[(shark["sex_"] == '.')] 

,year,type,country,activity,sex_,fatal_YN
5437,1908.0,Sea Disaster,PAPUA NEW GUINEA,.,.,Y


In [52]:
shark_cop.loc[(shark_cop["Sex "] == '.')]#NAN

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
5437,1908.06.02.R,Reported 02-Jun-1908,1908.0,Sea Disaster,PAPUA NEW GUINEA,New Britain,Matupi,.,NaN,.,...,Allegedly a 33-foot shark,"Taranaki Herald, 6/2/1908",1908.06.02.R-Matupi.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1908.06.02.R,1908.06.02.R,866.0,NaN,NaN


In [53]:
#En base a lo analizado tomo decision
shark['sex_'].replace(['M ', 'N', 'lli' ],'M', inplace=True)
shark['sex_'].replace('.',np.nan, inplace=True)

In [54]:
#FILTRO DE PAISES! 

In [55]:
dict_countries = dict(shark.country.value_counts())



In [56]:
#Filtramos los paises, por numero de incidentes. Convertimos 'shark.country.value_counts' en un diccionario, tenemos Key (paiese) y values (numero de incidentes), si el value es mayor que el numero propusto, reornara un 'True'. Nos quedara un dictionary con con paises con paises que sean True si cumple condición o false si no la cumple. 
filtro_1 = dict(shark.country.value_counts() > 50)

In [57]:
#Filtramos los paiese en los que el 'value' sea True, que son los que nos interesan. y tenemos una lista.
#Se puede hacer un un paso filtro uno y dos pero mejor en dos pasos para visualizar mejor.
filtro_2 = [k for k,v in filtro_1.items() if v == True]


In [58]:
#Tambien se pueden clasificar dentro de cada uno si son Fatales or not 

filtro_3 = {k:value for k,v in filtro_1.items() if v == True for key,value in dict_countries.items() if key == k }


In [59]:
#Creamos un Data Frame, con dos columnas de paises he incidencias.
filtro_4 = {'country':list(filtro_3.keys()),'incidents':list(filtro_3.values())}

In [60]:
filtro_5 = pd.DataFrame(filtro_4)

In [61]:
#inci_USA_Y = shark.loc[(shark["country"] == 'USA') & (shark["fatal_YN"]=="Y")]

In [62]:
#inci_USA_N = shark.loc[(shark["country"] == 'USA') & (shark["fatal_YN"]=="N")

In [63]:
#tupla = (inci_USA_N.shape)[0] - daria valor 1837 - numero de invidentes 'No fatales'

In [65]:
shark_1 = shark.copy()

In [66]:
regroup_1 = shark_1.groupby(['fatal_YN'])['country'].value_counts()

In [67]:
regroup_2 = regroup_1 .unstack().T
regroup_2.columns

Index(['N', 'Y'], dtype='object', name='fatal_YN')

In [68]:
regroup_3 = regroup_2.reset_index()

In [69]:
regroup_3.info

<bound method DataFrame.info of fatal_YN            country     N    Y
0               PHILIPPINES   NaN  1.0
1                     TONGA   1.0  2.0
2         ADMIRALTY ISLANDS   1.0  NaN
3                    AFRICA   NaN  1.0
4                   ALGERIA   NaN  1.0
..                      ...   ...  ...
196                 VIETNAM  13.0  2.0
197             WEST INDIES   NaN  2.0
198           WESTERN SAMOA   NaN  1.0
199                   YEMEN   NaN  2.0
200                  YEMEN    1.0  5.0

[201 rows x 3 columns]>

In [70]:
regroup_3.fillna(0)

fatal_YN,country,N,Y
0,PHILIPPINES,0.0,1.0
1,TONGA,1.0,2.0
2,ADMIRALTY ISLANDS,1.0,0.0
3,AFRICA,0.0,1.0
4,ALGERIA,0.0,1.0
...,...,...,...
196,VIETNAM,13.0,2.0
197,WEST INDIES,0.0,2.0
198,WESTERN SAMOA,0.0,1.0
199,YEMEN,0.0,2.0


In [71]:
regroup_3['incidentes'] = regroup_3.fillna(0)['N']+regroup_3.fillna(0)['Y']


In [72]:
regroup_3=regroup_3[regroup_3["incidentes"]>50]

In [75]:
regroup_4= regroup_3.set_index("country")


In [76]:
regroup_4.sort_values(['incidentes'], ascending=False,inplace=True) #, inplace=True ----- falta añadirle ---- 

In [77]:
regroup_4 = regroup_4.reset_index()

In [78]:
regroup_4

fatal_YN,country,N,Y,incidentes
0,USA,1837.0,186.0,2023.0
1,AUSTRALIA,921.0,283.0,1204.0
2,SOUTH AFRICA,407.0,106.0,513.0
3,PAPUA NEW GUINEA,74.0,56.0,130.0
4,NEW ZEALAND,91.0,24.0,115.0
5,BAHAMAS,94.0,10.0,104.0
6,BRAZIL,64.0,38.0,102.0
7,MEXICO,35.0,43.0,78.0
8,FIJI,39.0,22.0,61.0
9,REUNION,31.0,29.0,60.0


In [83]:
regroup_4.columns = ['country','DEAD','survived','incidents']

ValueError: Length mismatch: Expected axis has 3 elements, new values have 4 elements

In [80]:
regroup_4 = regroup_4.set_index("country")

In [82]:
regroup_4

,DEAD,survived,incidents
country,,,
USA,1837.0,186.0,2023.0
AUSTRALIA,921.0,283.0,1204.0
SOUTH AFRICA,407.0,106.0,513.0
PAPUA NEW GUINEA,74.0,56.0,130.0
NEW ZEALAND,91.0,24.0,115.0
BAHAMAS,94.0,10.0,104.0
BRAZIL,64.0,38.0,102.0
MEXICO,35.0,43.0,78.0
FIJI,39.0,22.0,61.0


In [81]:
regroup_4.to_csv('incidents_by_countries.csv', index=False)

In [79]:
shark

,year,type,country,activity,sex_,fatal_YN
0,2018.0,Boating,USA,Paddling,F,N
1,2018.0,Unprovoked,USA,Standing,F,N
2,2018.0,Invalid,USA,Surfing,M,N
3,2018.0,Unprovoked,AUSTRALIA,Surfing,M,N
4,2018.0,Provoked,MEXICO,Free diving,M,N
...,...,...,...,...,...,...
6297,0.0,Unprovoked,AUSTRALIA,Diving,M,Y
6298,0.0,Unprovoked,AUSTRALIA,Pearl diving,M,Y
6299,0.0,Unprovoked,USA,Swimming,M,Y
6300,0.0,Unprovoked,PANAMA,NaN,M,Y


In [80]:
#segunda hipotesis 

In [81]:
shark_2 = shark.copy()

In [117]:
shark_2

,year,type,country,activity,sex_,fatal_YN
0,2018.0,Boating,USA,Paddling,F,N
1,2018.0,Unprovoked,USA,Standing,F,N
2,2018.0,Invalid,USA,Surfing,M,N
3,2018.0,Unprovoked,AUSTRALIA,Surfing,M,N
4,2018.0,Provoked,MEXICO,Free diving,M,N
...,...,...,...,...,...,...
6297,0.0,Unprovoked,AUSTRALIA,Diving,M,Y
6298,0.0,Unprovoked,AUSTRALIA,Pearl diving,M,Y
6299,0.0,Unprovoked,USA,Swimming,M,Y
6300,0.0,Unprovoked,PANAMA,NaN,M,Y


In [110]:
hipotesis_2 = group_2.unstack()

In [111]:
hipotesis_2

fatal_YN,N,Y
sex_,,
F,468,107
M,3484,1174


In [112]:
hipotesis_2.reset_index()

fatal_YN,sex_,N,Y
0,F,468,107
1,M,3484,1174


In [113]:
hipotesis_2.index

Index(['F', 'M'], dtype='object', name='sex_')

In [114]:
hipotesis_2.columns = ['dead','injured']

In [115]:
hipotesis_2

,dead,injured
sex_,,
F,468,107
M,3484,1174


In [116]:
hipotesis_2.to_csv('incidents_by_gender.csv', index=False)